In [ ]:
%pip install -Uqq langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.chat_models import ChatPerplexity
from google.colab import userdata

In [ ]:
messages = [
    SystemMessage(content="Translate the following from English into Chinese:"),
    HumanMessage(content="Welcome to LLM application development!"),
]

model = ChatPerplexity(
    temperature=0, pplx_api_key=userdata.get('pplx_api_key'), model="llama-3.1-8b-instruct"
)

result = model.invoke(messages)
print(result)

content='欢迎来到LLM应用开发！' additional_kwargs={} response_metadata={} id='run-ef8d8875-264c-41f3-91cc-2e05e7bcc688-0'


In [ ]:
stream = model.stream(messages)
for response in stream:
    print(response.content, end="|", flush=True)

欢|迎来|到LL|M应用|开发！||

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Translate the following from English into Chinese:"),
        ("user", "{text}")
    ]
)

chain = prompt_template | model
result = chain.invoke({"text":"Welcome to LLM application development!"})
print(result)

content='欢迎来到LLM应用开发！' additional_kwargs={} response_metadata={} id='run-63c5301e-5cf5-40ef-9527-a5579562181d-0'


In [ ]:
messages = prompt_template.invoke({"text":"Welcome to LLM application development!"})
print(messages)

messages=[SystemMessage(content='Translate the following from English into Chinese:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Welcome to LLM application development!', additional_kwargs={}, response_metadata={})]


In [ ]:
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Translate the following from English into Chinese:"),
        ("user", "{text}")
    ]
)

parser = StrOutputParser()

chain = prompt_template | model | parser
result = chain.invoke({"text":"Welcome to LLM application development!"})
print(result)

欢迎来到LLM应用开发！


In [ ]:
model = ChatPerplexity(
    temperature=0, pplx_api_key=userdata.get('pplx_api_key'), model="llama-3.1-sonar-huge-128k-online"
)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Work(BaseModel):
    title: str = Field(description="Title of the work")
    description: str = Field(description="Description of the work")


parser = JsonOutputParser(pydantic_object=Work)

prompt = PromptTemplate(
    template="列举3部{author}的作品。\n{format_instructions}",
    input_variables=["author"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
result = chain.invoke({"author": "老舍"})
print(result)

[{'title': '骆驼祥子', 'description': '老舍笔下的祥子来自乡间，带着中国农村破败凋敝的大背景，也带着农民的质朴和固执。当他认准了拉车这一行，他就成了车迷，一心想买上自己拉车的骆驼。'}, {'title': '茶馆', 'description': '这部话剧真实地记述了北平沦陷后的畸形世态，形象地描摹了日寇侵占下的北京市民的生活和思想。'}, {'title': '四世同堂', 'description': '作品以祁家四世同堂的生活为主线，辅以小学圈胡同各色人等的荣辱浮沉、生死存亡，真实地记述了北平沦陷后的畸形世态。'}]


In [ ]:
messages = prompt.invoke({"author": "老舍"})
print(messages)

text='列举3部老舍的作品。\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "Title of the work", "title": "Title", "type": "string"}, "description": {"description": "Description of the work", "title": "Description", "type": "string"}}, "required": ["title", "description"]}\n```'
